In [ ]:
#Будут вопросы, пишите valencia13@mail.ru

import glob, re
import numpy as np
import pandas as pd
import xgboost as xgb
import gc
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, timedelta
import dateutil
import time
from catboost import CatBoostRegressor
import lightgbm as lgb
import time
from math import sqrt, pow as pw
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
# Определим типы колонок для экономии памяти
dtypes = {
    'transaction_date': str,
    'atm_address': str,
    'country': str,
    'city': str,
    'amount': np.float32,
    'currency': np.float32,
    'mcc': str,
    'customer_id': str,
    'pos_address': str,
    'atm_address': str,
    'pos_adress_lat': np.float32,
    'pos_adress_lon': np.float32,
    'pos_address_lat': np.float32,
    'pos_address_lon': np.float32,
    'atm_address_lat': np.float32,
    'atm_address_lon': np.float32,
    'home_add_lat': np.float32,
    'home_add_lon': np.float32,
    'work_add_lat': np.float32,
    'work_add_lon': np.float32,
}

# для экономии памяти будем загружать только часть атрибутов транзакций
usecols_train = ['customer_id','transaction_date','amount','country', 'city', 'currency', 'mcc', 'pos_adress_lat', 'pos_adress_lon', 'atm_address_lat', 'atm_address_lon','home_add_lat','home_add_lon','work_add_lat','work_add_lon']
usecols_test = ['customer_id','transaction_date','amount','country', 'city', 'currency', 'mcc', 'pos_address_lat', 'pos_address_lon', 'atm_address_lat', 'atm_address_lon']

train = pd.read_csv('train_set.csv', dtype = dtypes, usecols = usecols_train)
train.rename(columns = {'pos_adress_lat': 'pos_address_lat', 'pos_adress_lon': 'pos_address_lon'}, inplace = True)

test = pd.read_csv('test_set.csv', dtype = dtypes, usecols = usecols_test)
submission = pd.DataFrame(test['customer_id'].unique(), columns = ['_ID_'])

# соединяем test/train в одном DataFrame
train['is_train'] = np.int32(1)
test['is_train'] = np.int32(0)
data = pd.concat([train, test])

del train, test

In [ ]:
double_home_la = data.groupby(['customer_id'])['home_add_lat'].unique()
double_home_lo = data.groupby(['customer_id'])['home_add_lon'].unique()

In [ ]:
doub_h_la = []
for i in zip(double_home_la.index, double_home_la.values):
    if len(i[1]) >1:
        doub_h_la.append(i[0])
doub_h_lo = []
for i in zip(double_home_lo.index, double_home_lo.values):
    if len(i[1]) >1:
        doub_h_lo.append(i[0])


In [ ]:
double_work_la = data.groupby(['customer_id'])['work_add_lat'].unique()
double_work_lo = data.groupby(['customer_id'])['work_add_lon'].unique()

In [ ]:
doub_w_la = []
for i in zip(double_work_la.index, double_work_la.values):
    if len(i[1]) >1:
        doub_w_la.append(i[0])
doub_w_lo = []
for i in zip(double_work_lo.index, double_work_lo.values):
    if len(i[1]) >1:
        doub_w_lo.append(i[0])

In [ ]:
print(len(doub_h_la), len(doub_h_lo), len(doub_w_la), len(doub_w_lo))
del_us = set(doub_h_la + doub_h_lo + doub_w_la + doub_w_lo)
len(del_us)

In [ ]:
for i in del_us:
    data = data[data.customer_id != i]

In [ ]:
len(data.customer_id.unique())

In [ ]:
mcc = pd.read_csv('mcc.csv', header=None)
mcc.columns = ['mcc', 'cat', 'group']
# mcc

In [ ]:
data['mcc'] = data['mcc'].apply(lambda x: int(''.join(str(x).split(','))))
data['mcc'] = pd.to_numeric(data['mcc'])
data = data.join(mcc.set_index('mcc'), on='mcc', rsuffix='_2')
data.head(1)

In [ ]:
# data[['atm_address_lat', 'pos_address_lat', 'atm_address_lon', 'pos_address_lon']] = data[['atm_address_lat', 'pos_address_lat', 'atm_address_lon', 'pos_address_lon']].fillna(0)

data['ter_lat'] = data['atm_address_lat'].fillna(data['pos_address_lat'])
data['ter_lon'] = data['atm_address_lon'].fillna(data['pos_address_lon'])

data['ter_lat'] = data['ter_lat'].apply(lambda x: round(x,3))
data['ter_lon'] = data['ter_lon'].apply(lambda x: round(x,3))

data['atm_address_lat'] = data['atm_address_lat'].notnull()*1
data['pos_address_lat'] = data['pos_address_lat'].notnull()*1

data['atm_address_lon'] = data['atm_address_lon'].notnull()*1
data['pos_address_lon'] = data['pos_address_lon'].notnull()*1#.apply(lambda x: not(x.isnull())*1)

In [ ]:
# data['toh'] = data.apply(lambda x: ((x.ter_lat - x.home_add_lat)**2 + (x.ter_lon - x.home_add_lon)**2)**(1/2), axis=1)
# data['tow'] = data.apply(lambda x: ((x.ter_lat - x.work_add_lat)**2 + (x.ter_lon - x.work_add_lon)**2)**(1/2), axis=1)

In [ ]:
dates = pd.read_csv('dates.csv', header=None)

data.transaction_date = pd.to_datetime(data.transaction_date)


In [ ]:
data = data[(data['transaction_date'].notnull())]
datess = []
for i in range(12):
    datess.append(set(map(int, dates[i][0].split(','))))
datess


In [ ]:
data['week'] = data['transaction_date'].apply(lambda x: 0 if x.day in datess[x.month-1] else 1)


In [ ]:
print(data.ter_lat.isnull().sum(), data.ter_lon.isnull().sum(), data.shape)
data = data[np.isfinite(data['ter_lat'])]
# data = data[np.isfinite(data['work_add_lat'])]

data.shape

In [ ]:
# fig, ax = plt.subplots(figsize=(8,4))
# ax.plot(data['toh'], marker='.', linestyle='None')
# fig.autofmt_xdate()
# plt.title('toh')

In [ ]:
mimeco=data.groupby('customer_id')['amount'].agg(['count'])
# mimeco=data.groupby('customer_id')['toh'].agg(['min', 'mean', 'median', 'count'])
data = data.join(mimeco, on='customer_id', rsuffix='_2')

# mimeco=data.groupby('customer_id')['tow'].agg(['min'])
# # mimeco=data.groupby('customer_id')['tow'].agg(['min', 'mean', 'median', 'count'])

# data = data.join(mimeco, on='customer_id', rsuffix='_w')
mass = pd.DataFrame(data.groupby(['ter_lon'])['amount'].count())
mass.columns = ['mass']
data = data.join(mass, on='ter_lon', rsuffix='_2')  
# print(mass[mass.amount > 1000].shape)
# masss = set(mass[mass.amount > 1000]['ter_lon'])


In [ ]:
for i in ['ter_lat','ter_lon']:
    data[i+'_2'] = data[i].apply(lambda x: round(x, 2))
    data[i+'_1'] = data[i].apply(lambda x: round(x, 1))
    

mass1 = pd.DataFrame(data.groupby(['ter_lon_1', 'ter_lat_1'])['amount'].count()).reset_index()
mass2 = pd.DataFrame(data.groupby(['ter_lon_2', 'ter_lat_2'])['amount'].count()).reset_index()

data = pd.merge(data, mass1,  how='inner', left_on=['ter_lon_1', 'ter_lat_1'], right_on = ['ter_lon_1', 'ter_lat_1'])
data = pd.merge(data, mass2,  how='inner', left_on=['ter_lon_2', 'ter_lat_2'], right_on = ['ter_lon_2', 'ter_lat_2'])
data.head(100)

In [ ]:
# data[data['min']>0.02].shape
data['weekday'] = data['transaction_date'].apply(lambda x: x.weekday())
data['day'] = data['transaction_date'].apply(lambda x: x.weekday())
data['weekend'] = data['weekday'].apply(lambda x: ((x==5) or (x==6))*1)
data['month'] = data['transaction_date'].apply(lambda x: x.month)


In [ ]:
coord = pd.read_csv('coord_all.csv')
data = data.join(coord.set_index('cus'), on='customer_id', rsuffix='_2')
data.shape

In [ ]:
print(data.shape)

data = data[np.isfinite(data['lat1'])]

data.shape

In [ ]:
data['currency'] = data['currency'].fillna(-1).astype(np.int32)
# data['mcc'] = data['mcc'].apply(lambda x: int(x.replace(',', ''))).astype(np.int32)
data['city'] = data['city'].factorize()[0].astype(np.int32)
data['country'] = data['country'].factorize()[0].astype(np.int32)

# удаляем транзакции без даты
data.drop(data[data['transaction_date'].isnull()].index, axis = 0, inplace = True)



In [ ]:
# from tsfresh import extract_features
# from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
# from tsfresh import select_features
# from tsfresh.utilities.dataframe_functions import impute

# settings=MinimalFCParameters()
# settings['number_cwt_peaks'] = ComprehensiveFCParameters()['number_cwt_peaks']
# settings['number_peaks'] = ComprehensiveFCParameters()['number_peaks']
# settings['cid_ce'] = ComprehensiveFCParameters()['cid_ce']
# # settings['agg_linear_trend'] = ComprehensiveFCParameters()['agg_linear_trend']
# settings['agg_autocorrelation'] = ComprehensiveFCParameters()['agg_autocorrelation']
# settings['c3'] = ComprehensiveFCParameters()['c3']
# settings


In [ ]:
# data_am = pd.DataFrame(data.groupby(['customer_id', 'transaction_date'])['amount'].median()).reset_index()

In [ ]:
# extracted_features = extract_features(data, column_id="customer_id",\
#                                       column_sort="transaction_date", column_value="ter_lon", default_fc_parameters=settings, n_jobs=4)
# impute(extracted_features)
# # features_filtered = select_features(extracted_features, target)
# data = data.join(extracted_features, on='customer_id')

# # del extracted_features; gc.collect()

In [ ]:
# extracted_features.to_csv('extracted_features_lon.csv')
ext_f = pd.read_csv('extracted_features.csv')
# ext_lat = pd.read_csv('extracted_features_lat.csv')
# ext_lon = pd.read_csv('extracted_features_lon.csv')
ext_f = ext_f.set_index('id')
# ext_lat = ext_lat.set_index('id')
# ext_lon = ext_lon.set_index('id')

data = data.join(ext_f, on='customer_id')
# data = data.join(ext_lat, on='customer_id')
# data = data.join(ext_lon, on='customer_id')

In [ ]:
# print(data.shape)

# data['is_atm'] = (~data['atm_address_lat'].isnull()).astype(np.int32)
# data['is_pos'] = (~data['pos_address_lat'].isnull()).astype(np.int32)

# data['address_lat'] = data['atm_address_lat'].fillna(0) + data['pos_address_lat'].fillna(0)
# data['address_lon'] = data['atm_address_lon'].fillna(0) + data['pos_address_lon'].fillna(0)

# data.drop(['atm_address_lat','atm_address_lon','pos_address_lat','pos_address_lon'], axis = 1, inplace = True)

# # удалим транзакции без адреса
# data.drop(data[((data['address_lon'] == 0) & (data['address_lon'] == 0))].index, axis = 0, inplace = True)

# print(data.shape)


In [ ]:
lat = data['home_add_lat'] - data['ter_lat']
lon = data['home_add_lon'] - data['ter_lon']
data['is_home'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
data['has_home'] = (~data['home_add_lon'].isnull()).astype(np.int32)

lat = data['work_add_lat'] - data['ter_lat']
lon = data['work_add_lon'] - data['ter_lon']
data['is_work'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
data['has_work'] = (~data['work_add_lon'].isnull()).astype(np.int32)

# data.drop(['work_add_lat','work_add_lon','home_add_lat','home_add_lon'], axis = 1, inplace = True)

data['address'] = data['ter_lat'].apply(lambda x: "%.02f" % x) + ';' + data['ter_lon'].apply(lambda x: "%.02f" % x)
data['address'] = data['address'].factorize()[0].astype(np.int32)

data = data.merge(data.groupby(['customer_id','address'])['amount'].count().reset_index(name = 'tx_cust_addr'), how = 'left')
data['tx_cust_addr'] = data['tx_cust_addr'].astype(np.int32)

# какая часть транзакций клиента приходится на данный адрес
data['ratio1'] = data['tx_cust_addr'] / data['count']

In [ ]:
# def add_noise(series, noise_level):
#     return series * (1 + noise_level * np.random.randn(len(series)))

# def target_encode(trn_series=None, 
#                   tst_series=None, 
#                   target=None, 
#                   min_samples_leaf=1, 
#                   smoothing=1,
#                   noise_level=0):
#     """
#     Smoothing is computed like in the following paper by Daniele Micci-Barreca
#     https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
#     trn_series : training categorical feature as a pd.Series
#     tst_series : test categorical feature as a pd.Series
#     target : target data as a pd.Series
#     min_samples_leaf (int) : minimum samples to take category average into account
#     smoothing (int) : smoothing effect to balance categorical average vs prior  
#     """ 
#     assert len(trn_series) == len(target)
#     assert trn_series.name == tst_series.name
#     temp = pd.concat([trn_series, target], axis=1)
#     # Compute target mean 
#     averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
#     # Compute smoothing
#     smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
#     # Apply average function to all target data
#     prior = target.mean()
#     # The bigger the count the less full_avg is taken into account
#     averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
#     averages.drop(["mean", "count"], axis=1, inplace=True)
#     # Apply averages to trn and tst series
#     ft_trn_series = pd.merge(
#         trn_series.to_frame(trn_series.name),
#         averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
#         on=trn_series.name,
#         how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
#     # pd.merge does not keep the index so restore it
#     ft_trn_series.index = trn_series.index 
#     ft_tst_series = pd.merge(
#         tst_series.to_frame(tst_series.name),
#         averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
#         on=tst_series.name,
#         how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
#     # pd.merge does not keep the index so restore it
#     ft_tst_series.index = tst_series.index
#     return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

# trn, sub = target_encode(data[data.is_train==1]['mcc'], 
#                          data[data.is_train==0]['mcc'], 
#                          target=data[data.is_train==1].is_home, 
#                          min_samples_leaf=100, #65 #70 не зашло
#                          smoothing=10,
#                          noise_level=0.01)
# sub.head(100)
# # # ind = len(train)
# # # full = pd.concat([train_full, test_full]).reset_index(drop=True)
# # # doub = full.groupby(['norm'])['Label'].mean()
# # # print(doub)

# # # train_full = train_full.join(doub, on='norm', rsuffix='_d')
# # # test_full = test_full.join(doub, on='norm', rsuffix='_d')

# # full = pd.concat([train_full, test_full]).fillna(0).reset_index(drop=True)
# # doub = full.groupby(['norm'])['Label'].mean()
# # print(doub)

# # train_full = train_full.join(doub, on='norm', rsuffix='_d')
# # test_full = test_full.join(doub, on='norm', rsuffix='_d')

# # train_full.rename(columns={'Label_d':'Mean'}, inplace=True)
# # test_full.rename(columns={'Label':'Mean'}, inplace=True)


In [ ]:
# start_time = time.time()
# k = 0
# innlalol = []
# for i in ['90ea490b1609f862033a44b615690176', 'daa79438cba5089c7fd9ce7c4b1850bd', '1730b4288ddf5c690f31c0e463a7dec7']:
    
#     da = data[data.customer_id == i]
#     melat = da['ter_lat'].median()
#     melon = da['ter_lon'].median()
#     inner_lat = da['ter_lat'].values
#     inner_lon = da['ter_lon'].values
#     lat = inner_lat - melat
#     lon = inner_lon - melon
#     n = [sqrt(pw(la, 2) + pw(lo, 2)) for la, lo in zip(lat,lon)]
#     innlalol.extend(n)


# print(k, time.time() - start_time)

In [ ]:
# start_time = time.time()
# k = 0
# innlalo = []

# for i in data.customer_id.unique():
#     da = data[data.customer_id == i]
#     melat = da['ter_lat'].median()
#     melon = da['ter_lon'].median()
#     inner_lat = da['ter_lat'].values
#     inner_lon = da['ter_lon'].values
#     lat = inner_lat - melat
#     lon = inner_lon - melon
#     n = [sqrt(pw(la,2) + pw(lo, 2)) for la, lo in zip(lat,lon)]
#     innlalo.extend(n)
#     k += 1
#     if k%100==0:
#         print(k, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# inll = pd.DataFrame(innlalo)
# inll.columns = ['innlalo']
# inll.to_csv('innlalo.csv', index=False)
# inll.head()
# cou_mcc = pd.DataFrame(data[(data.is_train==1)].groupby('mcc')[['amount']].count()).reset_index()
# cou_mcc.sort_values(by=['amount'], ascending=False)
# mmccc = []
# for i in range(cou_mcc.shape[0]):
#     if cou_mcc['amount'].iloc[i]>8000:
#         mmccc.append(cou_mcc['mcc'].iloc[i])
# mmccc
inll = pd.read_csv('innlalo.csv')
inll.head()

In [ ]:
data = data.reset_index(drop=True)
data['innlalo'] = inll['innlalo']

In [ ]:
data.head()

# data = data.set_index('mcc')
# # all_mcc_coord = []

# for i in mmccc:
#     exec('inner{} = data[["ter_lat", "ter_lon"]].loc[i]'.format(i))
#     exec('inner{} = inner{}.drop_duplicates()'.format(i, i))

# data = data.reset_index()


In [ ]:
# # # def get_nn_eps(df, data, eps, mode):
# # #     from sklearn.neighbors import NearestNeighbors
# # #     neigh = NearestNeighbors(radius=eps, n_jobs=4)
# # #     customer_id_dct_te=df.groupby(['customer_id']).count().index
# # #     nn_eps_te=[]
# # # #     print(customer_id_dct_te)
# # #     for i,item in enumerate(customer_id_dct_te):
# # #         if mode=='train':
# # #             b=np.vstack((data[:i],data[i+1:]))
# # #         else:
# # #             b=data
# # #         neigh.fit(b)
# # #         vv = neigh.radius_neighbors(df[df['customer_id']==item][["ter_lat", "ter_lon"]])
# # # #         for j in neigh.radius_neighbors(df[df['customer_id']==item][["ter_lat", "ter_lon"]])[0]:
# # # #             print(j.shape[0])
# # # #             nn_eps_te.append(j.shape[0])
# # #     return vv.shape
# lli = [] 
# for i in mmccc:
#     exec('lli.append(({}, inner{}.shape[0]))'.format(i, i))
# set(lli)

In [ ]:
# data_coor = data[['ter_lat', 'ter_lon']]
# print(data_coor.shape)
# data_coor = data_coor.drop_duplicates()
# data_coor.shape


In [ ]:
# import sys
# from io import StringIO

# #remember to restore the original stdout!

# for mcc in lli:
#     data_coor[mcc[0]] = -1
#     inn = []
#     print((mcc[0]))
#     buffer = StringIO()
#     sys.stdout = buffer
#     for coor in range(data_coor.shape[0]):
#         n = 0
#         for mcc_coor in range(mcc[1]):
#             exec('print(inner{}["ter_lat"].iloc[mcc_coor])'.format(mcc[0]))
#             latt = float(buffer.getvalue()[:-1])
#             buffer = StringIO()
#             sys.stdout = buffer
#             exec('print(inner{}["ter_lon"].iloc[mcc_coor])'.format(mcc[0]))
#             lonn = float(buffer.getvalue()[:-1])
#             buffer = StringIO()
#             sys.stdout = buffer
#             lat = data_coor['ter_lat'].iloc[coor] - latt
#             lon = data_coor['ter_lon'].iloc[coor] - lonn
#             if (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02):
#                 n+=1
# #         data_coor[mcc[0]].iloc[coor] = n
#         inn.append(n)
#     data_coor[mcc[0]] = inn

#     sys.stdout = sys.__stdout__


In [ ]:
# #inner5814
# import time
# from math import sqrt, pow as pw
# start_time = time.time()

# # data_coor['inner5814'] = -1
# inn = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5814.shape[0]):
#         lat = data_coor[coor][0] - inner5814[mcc_coor][0]
#         lon = data_coor[coor][1] - inner5814[mcc_coor][1]
#         if (math.sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn.append(n)
#     if coor%1000==0:
#         print(coor, time.time() - start_time)
# # data_coor['inner5814'] = inn
# print("--- %s seconds ---" % (time.time() - start_time))
# #         lat = data_coor['ter_lat'].iloc[coor] - inner5814['ter_lat'].iloc[mcc_coor]
# #         lon = data_coor['ter_lon'].iloc[coor] - inner5814['ter_lon'].iloc[mcc_coor]

# # 0 0.02087688446044922
# # 1000 21.12002658843994
# # 2000 41.68081474304199

# # 0 0.014701128005981445
# # 1000 13.027771472930908
# # 2000 26.112202167510986

# # 0 0.015043020248413086
# # 1000 12.524210453033447
# # 2000 24.834967851638794

In [ ]:
# inner5814 = inner5814.as_matrix()
# data_coor = data_coor.as_matrix()

In [ ]:
# # data = data.set_index('mcc')
# # # all_mcc_coord = []

# # for i in mmccc:
# #     exec('inner{} = data[["ter_lat", "ter_lon"]].loc[i]'.format(i))
# #     exec('inner{} = inner{}.drop_duplicates()'.format(i, i))

# # data = data.reset_index()

# data_coor = data[['ter_lat', 'ter_lon']]
# print(data_coor.shape)
# data_coor = data_coor.drop_duplicates()
# data_coor.shape

# data_lat = data_coor['ter_lat'].values
# data_lon = data_coor['ter_lon'].values
# # inner5411_lat = inner5411['ter_lat'].values
# # inner5411_lon = inner5411['ter_lon'].values
# data_lat

In [ ]:
# #inner5411

# start_time = time.time()

# inn1 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5411.shape[0]):
#         lat = data_lat[coor] - inner5411_lat[mcc_coor]
#         lon = data_lon[coor] - inner5411_lon[mcc_coor]
#         if (math.sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn1.append(n)
#     if coor%1000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))


# # #inner5411

# # start_time = time.time()

# # inn1 = []
# # for coor in range(data_coor.shape[0]):   
# #     n = 0
# #     for mcc_coor in range(inner5411.shape[0]):
# #         lat = data_coor[coor][0] - inner5411[mcc_coor][0]
# #         lon = data_coor[coor][1] - inner5411[mcc_coor][1]
# #         if (math.sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
# #             n+=1
# #     inn1.append(n)
# #     if coor%1000==0:
# #         print(coor, time.time() - start_time)
# # print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# inner6011_lat = inner6011['ter_lat'].values
# inner6011_lon = inner6011['ter_lon'].values

# start_time = time.time()

# inn6011 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner6011.shape[0]):
#         lat = data_lat[coor] - inner6011_lat[mcc_coor]
#         lon = data_lon[coor] - inner6011_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn6011.append(n)
#     if coor%1000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# inner4111_lat = inner4111['ter_lat'].values
# inner4111_lon = inner4111['ter_lon'].values

# start_time = time.time()

# inn4111 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner4111.shape[0]):
#         lat = data_lat[coor] - inner4111_lat[mcc_coor]
#         lon = data_lon[coor] - inner4111_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn4111.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# inner5261_lat = inner5261['ter_lat'].values
# inner5261_lon = inner5261['ter_lon'].values

# start_time = time.time()

# inn5261 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5261.shape[0]):
#         lat = data_lat[coor] - inner5261_lat[mcc_coor]
#         lon = data_lon[coor] - inner5261_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5261.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# inner5311_lat = inner5311['ter_lat'].values
# inner5311_lon = inner5311['ter_lon'].values

# start_time = time.time()

# inn5311 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5311.shape[0]):
#         lat = data_lat[coor] - inner5311_lat[mcc_coor]
#         lon = data_lon[coor] - inner5311_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5311.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# inner5331_lat = inner5331['ter_lat'].values
# inner5331_lon = inner5331['ter_lon'].values

# start_time = time.time()

# inn5331 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5331.shape[0]):
#         lat = data_lat[coor] - inner5331_lat[mcc_coor]
#         lon = data_lon[coor] - inner5331_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5331.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# inner5499_lat = inner5499['ter_lat'].values
# inner5499_lon = inner5499['ter_lon'].values

# start_time = time.time()

# inn5499 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5499.shape[0]):
#         lat = data_lat[coor] - inner5499_lat[mcc_coor]
#         lon = data_lon[coor] - inner5499_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5499.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #  (5541, 4612),
# #  (5691, 2313),
# #  (5812, 7728),
# ##  (5814, 6360),
# #  (5912, 9642),
# #  (5921, 3894),
# #  (5977, 2192),
# #  (5999, 2899),
# ##  (6011, 87623)

# inner5541_lat = inner5541['ter_lat'].values
# inner5541_lon = inner5541['ter_lon'].values

# start_time = time.time()

# inn5541 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5541.shape[0]):
#         lat = data_lat[coor] - inner5541_lat[mcc_coor]
#         lon = data_lon[coor] - inner5541_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5541.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #  (5691, 2313),
# #  (5812, 7728),
# ##  (5814, 6360),
# #  (5912, 9642),
# #  (5921, 3894),
# #  (5977, 2192),
# #  (5999, 2899),
# ##  (6011, 87623)

# inner5691_lat = inner5691['ter_lat'].values
# inner5691_lon = inner5691['ter_lon'].values

# start_time = time.time()

# inn5691 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5691.shape[0]):
#         lat = data_lat[coor] - inner5691_lat[mcc_coor]
#         lon = data_lon[coor] - inner5691_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5691.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #  (5812, 7728),
# ##  (5814, 6360),
# #  (5912, 9642),
# #  (5921, 3894),
# #  (5977, 2192),
# #  (5999, 2899),
# ##  (6011, 87623)

# inner5812_lat = inner5812['ter_lat'].values
# inner5812_lon = inner5812['ter_lon'].values

# start_time = time.time()

# inn5812 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5812.shape[0]):
#         lat = data_lat[coor] - inner5812_lat[mcc_coor]
#         lon = data_lon[coor] - inner5812_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5812.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #  (5912, 9642),
# #  (5921, 3894),
# #  (5977, 2192),
# #  (5999, 2899),
# ##  (6011, 87623)

# inner5912_lat = inner5912['ter_lat'].values
# inner5912_lon = inner5912['ter_lon'].values

# start_time = time.time()

# inn5912 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5912.shape[0]):
#         lat = data_lat[coor] - inner5912_lat[mcc_coor]
#         lon = data_lon[coor] - inner5912_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5912.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #  (5921, 3894),
# #  (5977, 2192),
# #  (5999, 2899),
# ##  (6011, 87623)

# inner5921_lat = inner5921['ter_lat'].values
# inner5921_lon = inner5921['ter_lon'].values

# start_time = time.time()

# inn5921 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5921.shape[0]):
#         lat = data_lat[coor] - inner5921_lat[mcc_coor]
#         lon = data_lon[coor] - inner5921_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5921.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #  (5977, 2192),
# #  (5999, 2899),
# ##  (6011, 87623)

# inner5977_lat = inner5977['ter_lat'].values
# inner5977_lon = inner5977['ter_lon'].values

# start_time = time.time()

# inn5977 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5977.shape[0]):
#         lat = data_lat[coor] - inner5977_lat[mcc_coor]
#         lon = data_lon[coor] - inner5977_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5977.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #  (5999, 2899),
# ##  (6011, 87623)

# inner5999_lat = inner5999['ter_lat'].values
# inner5999_lon = inner5999['ter_lon'].values

# start_time = time.time()

# inn5999 = []
# for coor in range(data_coor.shape[0]):   
#     n = 0
#     for mcc_coor in range(inner5999.shape[0]):
#         lat = data_lat[coor] - inner5999_lat[mcc_coor]
#         lon = data_lon[coor] - inner5999_lon[mcc_coor]
#         if (sqrt(pw(lat,2) + pw(lon, 2)) <= 0.02):
#             n+=1
#     inn5999.append(n)
#     if coor%10000==0:
#         print(coor, time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# # data_coor = data[['ter_lat', 'ter_lon']]
# # print(data_coor.shape)
# # data_coor = data_coor.drop_duplicates()
# # data_coor['inn'] = inn
# # data_coor['inn1'] = inn1
# # data_coor = data_coor.reset_index(drop = True)
# # # data_coor = data_coor.set_index(['ter_lat', 'ter_lon'])
# # data_coor.head()


# data_coor = pd.read_csv('data_coor.csv')
# data_coor['i4111'] = inn4111
# data_coor['i5261'] = inn5261
# data_coor['i5311'] = inn5311
# data_coor['i5331'] = inn5331
# data_coor['i5499'] = inn5499
# data_coor['i5691'] = inn5691
# data_coor['i5812'] = inn5812
# data_coor['i5912'] = inn5912
# data_coor['i5921'] = inn5921
# data_coor['i5977'] = inn5977
# data_coor['i5999'] = inn5999
# data_coor['i6011'] = inn6011
# data_coor.head()

In [ ]:
# data_coor.to_csv('data_coor.csv', index=False)

data_coor = pd.read_csv('data_coor.csv')

In [ ]:
data = pd.merge(data, data_coor,  how='inner', left_on=['ter_lat', 'ter_lon'], right_on = ['ter_lat', 'ter_lon'])
data.head()

In [ ]:
# data.drop(['wd_home', 'wd_work', 'w_home', 'w_work', 'gw_home', 'gw_work', 'gwd_home', 'gwd_work'], axis = 1, inplace = True)

# data_mcc = data[(data.weekend==0)].groupby('mcc')[['is_home', 'is_work']].count()
# data_mcc.columns = ['w_home', 'w_work']
# data = data.join(data_mcc, on='mcc', rsuffix='_2')  

# data_mcc = data[(data.weekend==1)].groupby('mcc')[['is_home', 'is_work']].count()
# data_mcc.columns = ['wd_home', 'wd_work']
# data = data.join(data_mcc, on='mcc', rsuffix='_2')  

# data_mcc = data[(data.weekend==0)].groupby('group')[['is_home', 'is_work']].count()
# data_mcc.columns = ['gw_home', 'gw_work']
# data = data.join(data_mcc, on='group', rsuffix='_2')  

# data_mcc = data[(data.weekend==1)].groupby('group')[['is_home', 'is_work']].count()
# data_mcc.columns = ['gwd_home', 'gwd_work']
# data = data.join(data_mcc, on='group', rsuffix='_2')
# data.drop(['atm_address_lon', 'pos_address_lon'], axis = 1, inplace = True)



In [ ]:
# lat = data['home_add_lat'] - data['lat1']
# lon = data['home_add_lon'] - data['lon1']
# data['is_home1'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)

# lat = data['work_add_lat'] - data['lat1']
# lon = data['work_add_lon'] - data['lon1']
# data['is_work1'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)


# lat = data['home_add_lat'] - data['lat2']
# lon = data['home_add_lon'] - data['lon2']
# data['is_home2'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)

# lat = data['work_add_lat'] - data['lat2']
# lon = data['work_add_lon'] - data['lon2']
# data['is_work2'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)


# lat = data['home_add_lat'] - data['lat3']
# lon = data['home_add_lon'] - data['lon3']
# data['is_home3'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)

# lat = data['work_add_lat'] - data['lat3']
# lon = data['work_add_lon'] - data['lon3']
# data['is_work3'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)

# data.head()

In [ ]:
# # data['sssum_w'] = data['is_work1'] + data['is_work2'] + data['is_work3']
# # data['sssum_h'] = data['is_home1'] + data['is_home2'] + data['is_home3']

# lat = data['home_add_lat'] - data['ter_lat']
# lon = data['home_add_lon'] - data['ter_lon']
# data['is_home03'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)
# lat = data['work_add_lat'] - data['ter_lat']
# lon = data['work_add_lon'] - data['ter_lon']
# data['is_work03'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)

# lat = data['home_add_lat'] - data['lat1']
# lon = data['home_add_lon'] - data['lon1']
# data['is_home103'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)

# lat = data['work_add_lat'] - data['lat1']
# lon = data['work_add_lon'] - data['lon1']
# data['is_work103'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)


# lat = data['home_add_lat'] - data['lat2']
# lon = data['home_add_lon'] - data['lon2']
# data['is_home203'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)

# lat = data['work_add_lat'] - data['lat2']
# lon = data['work_add_lon'] - data['lon2']
# data['is_work203'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)


# lat = data['home_add_lat'] - data['lat3']
# lon = data['home_add_lon'] - data['lon3']
# data['is_home303'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)

# lat = data['work_add_lat'] - data['lat3']
# lon = data['work_add_lon'] - data['lon3']
# data['is_work303'] = ((np.sqrt((lat ** 2) + (lon ** 2)) <= 0.03)&(np.sqrt((lat ** 2) + (lon ** 2)) > 0.02)).astype(np.int32)

In [ ]:
# # ishw = data.groupby('customer_id')[['is_work', 'is_home', 'is_work1', 'is_home1', 'is_work2', 'is_home2', 'is_work3', 'is_home3',\
# #                 'is_work03', 'is_home03', 'is_work103', 'is_home103', 'is_work203', 'is_home203', 'is_work303', 'is_home303']].max()
# without = pd.DataFrame(data.groupby('customer_id')['is_home'].max())
# without.columns = ['without']
# data = data.join(without, on='customer_id', rsuffix='_2')  

# # ishw = data[(data.is_train==1)].groupby('customer_id')[['is_work', 'is_home', 'is_work1', 'is_home1', 'is_work2', 'is_home2', 'is_work3', 'is_home3']].max()

In [ ]:
# without = pd.DataFrame(data[data.is_train==1].groupby('customer_id')[['is_home', 'is_work']].max())

# ishw.describe()
# data.drop(['is_work', 'is_home', 'is_work1', 'is_home1', 'is_work2', 'is_home2', 'is_work3', 'is_home3', 'without', 'without_2'], axis = 1, inplace = True)

# data.head()

In [ ]:
# from sklearn.model_selection import train_test_split 
# from sklearn.metrics import mean_squared_error
# data_ca = data[(data.group != 'Отели и мотели')&(data.mcc != 5542)&(data.group != 'Транспорт')&(data.is_train==1)]

# # (data.group != 'Развлечения')

# cols = ['amount','currency', 'mcc', 'ter_lat','ter_lon', 'week', 'count', 'weekday', 'month',\
#        'lat1', 'lon1', 'cou1','lat2','lon2','cou2','lat3','lon3','cou3', 'is_home', 'is_work',\
#         'city','country', 'atm_address_lat','pos_address_lat','ratio1', 'tx_cust_addr', 'customer_id']
# data_cat = data_ca[cols]
# data_cat.shape, data_ca.shape

In [ ]:
# data_cat = data_cat.reset_index()
# x_tr,  x_vl = train_test_split(data_cat.customer_id.unique(), random_state=777, test_size=0.25)

# data_cat = data_cat.set_index('customer_id')
# tr = data_cat.loc[x_tr]
# vl = data_cat.loc[x_vl]
# tr = tr.reset_index(drop=True)
# vl = vl.reset_index(drop=True)
# tr.shape, vl.shape

In [ ]:
# import lightgbm as lgb

# # cols = [col for col in tr.columns if 'id' not in col and  'first_prch' not in col and 'date' not in col \
# #         and 'time' not in col and 'target' not in col and 'code' not in col]

# # cols = [col for col in tr.columns if col not in ['id', 'first_prch', \
# #     'date', 'code', 'time', 'target', 'level_0', 'index']]

# cols = ['amount','currency', 'mcc', 'ter_lat','ter_lon', 'week', 'count', 'weekday', 'month',\
#        'lat1', 'lon1', 'cou1','lat2','lon2','cou2','lat3','lon3','cou3', \
#         'city','country', 'atm_address_lat','pos_address_lat','ratio1', 'tx_cust_addr']

# # data_cat_tr = 
# # data_cat_vl = 

# # # print(data_cat_tr.shape)

# lgb_train = lgb.Dataset(tr[cols] , tr['is_home'])
# lgb_eval = lgb.Dataset(vl[cols] , vl['is_home'])

# # specify your configurations as a dict
# # params = {
# #     'task': 'train',
# #     'boosting_type': 'gbdt',
# #     'objective':'binary',
# #     'metric': {'auc'},
# #     'verbose': 0
# # }

# # print(data_cat_tr.shape, tr['target'].shape)
# # train
# # gbm = lgb.train(params,
# #                 lgb_train,
# #                 num_boost_round=20,
# #                 valid_sets=lgb_eval,
# #                 early_stopping_rounds=5)

In [ ]:
# params = {'max_depth': 7, 
#           'min_child_weight': 50, 
#           'bagging_fraction': 0.1, #0.3
#           'feature_fraction': 0.5,   #0.7
#           'boosting': 'dart',
#           'learning_rate': 0.1,
#           'lambda_l2': 3,
#           'objective': 'binary',
#           'metric': 'auc',
#           'data_random_seed': 777
#          }

# gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=100,
#                 valid_sets=lgb_eval,
#                 verbose_eval=10,
#                 early_stopping_rounds=50)

In [ ]:
# lgb_train = lgb.Dataset(tr[cols] , tr['is_work'])
# lgb_eval = lgb.Dataset(vl[cols] , vl['is_work'])

# params = {'max_depth': 7, 
#           'min_child_weight': 50, 
#           'bagging_fraction': 0.1, #0.3
#           'feature_fraction': 0.5,   #0.7
#           'boosting': 'dart',
#           'learning_rate': 0.1,
#           'lambda_l2': 3,
#           'objective': 'binary',
#           'metric': 'auc',
#           'data_random_seed': 777
#          }

# gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=100,
#                 valid_sets=lgb_eval,
#                 verbose_eval=10,
#                 early_stopping_rounds=50)

In [ ]:
# for i in ['lat1','lon1','lat2','lon2','lat3','lon3']:
#     data[i] = data[i].apply(lambda x: round(x, 2))
# data.head()
mccc = pd.DataFrame(data.groupby(['customer_id', 'mcc'])['amount'].count())
mccc.columns = ['mcc_cuc_am']
mccc = mccc.reset_index()
mccc.head()

In [ ]:
data = pd.merge(data, mccc,  how='inner', left_on=['customer_id', 'mcc'], right_on = ['customer_id', 'mcc'])


In [ ]:
cols = [col for col in data.columns if col.startswith('i') and col not in ['is_train', 'is_home', 'is_work']]
for col in cols:
    data['mcc_cuc_amii'+col] = data[col] - data['mcc_cuc_am']
#     data['ref'+col] = data['mcc_cuc_am']/data[col]
data['wonder'] = data['inn']+data['inn1']+data['i4111']+data['i5261']+data['i5311']+data['i5331']+data['i5499']+\
   data['i5691']+data['i5812']+data['i5912']+data['i5921']+data['i5977']+data['i5999']+data['i6011'] - 13*data['mcc_cuc_am']

In [ ]:
customers = data.groupby('customer_id')[['ter_lat', 'ter_lon']].agg(['mean', 'median'])
customers.columns = ['latme', 'latmed', 'lonme', 'lonmed']
customers.head()

In [ ]:
data = data.join(customers, on='customer_id')

In [ ]:
def _best(x):
    ret = None
    for col in ys:
        pred = ('pred:%s' % col)
        if pred in x:
            i = (x[pred].idxmax())
            if col == 'is_work':
                cols = [pred,'ter_lat','ter_lon', 'work_add_lat','work_add_lon']

            elif col == 'is_home':
                cols = [pred,'ter_lat','ter_lon','home_add_lat','home_add_lon']



#             if col in x:
#                 cols.append(col)
            tmp = x.loc[i,cols]
            tmp.rename({
                'ter_lat':'%s:add_lat' % col,
                'ter_lon':'%s:add_lon' % col,
            }, inplace = True)
            if ret is None:
                ret = tmp
            else:
                ret = pd.concat([ret, tmp])
    return ret

def predict_proba(dt, ys = ['is_home', 'is_work']):
    for col in ys:
        pred = ('pred:%s' % col)
        dt[pred] = model[col].predict(dt[xs])
    somefr = dt.groupby('customer_id').apply(_best).reset_index()
    if col == 'is_work':
        lat = somefr['work_add_lat'] - somefr['is_work:add_lat']
        lon = somefr['work_add_lon'] - somefr['is_work:add_lon']
        somefr['is_work'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
    elif col == 'is_home':
        lat = somefr['home_add_lat'] - somefr['is_home:add_lat']
        lon = somefr['home_add_lon'] - somefr['is_home:add_lon']
        somefr['is_home'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
    return somefr


def score(dt, ys = ['is_home', 'is_work']):
    dt_ret = predict_proba(dt, ys)
    mean = 0.0
    for col in ys:
        col_mean = dt_ret[col].mean()
        mean += col_mean
    if len(ys) == 2:
        mean = mean / len(ys)
    return mean

params = {'max_depth': 7, 
          'min_child_weight': 50, #50
          'bagging_fraction': 0.3, #0.7
          'feature_fraction': 0.7,   #0.7
          'boosting': 'dart',
          'learning_rate': 0.05,
          'lambda_l2': 3,
          'objective': 'binary',
#           'metric': 'auc',
          'data_random_seed': 777
         }

In [ ]:
# xs = ['amount','currency', 'mcc', 'ter_lat','ter_lon', 'week', 'count', 'weekday', 'month',\
#        'lat1', 'lon1', 'cou1','lat2','lon2','cou2','lat3','lon3','cou3', 'mass',\
#         'city','country', 'atm_address_lat','pos_address_lat','ratio1', 'tx_cust_addr']
xs = [col for col in data.columns if col not in ['address', 'atm_address_lon', 'cat', 'customer_id', 'group',\
    'has_home', 'has_work', 'is_home', 'is_train', 'is_work', 'pos_address_lon', 'transaction_date',\
        'home_add_lat', 'home_add_lon', 'work_add_lat', 'work_add_lon', 'day', 'weekend',\
        'ter_lat_2', 'ter_lat_1', 'ter_lon_2', 'ter_lon_1'] and not col.startswith('mcc_cuc_amiii')]

ys = ['is_home', 'is_work']

In [ ]:
# set(data.columns)
for i in (data[xs].columns):
    print(i)
# model0 = {
#     'is_home': gbm,
#     'is_work': gbm,
# }

In [ ]:
model = {}

from sklearn.model_selection import train_test_split 
data_ca = data[(data.group != 'Отели и мотели')&(data.mcc != 5542)&(data.group != 'Транспорт')]
# последовательно обучаем два классификатора
for col in ['is_home', 'is_work']:
    
    #выберем для обучение транзакции только тех клиентов из train, у которых хоть в одной транзакции указано место работы/жительства
    cust_train = data_ca[data_ca['is_train'] == 1].groupby('customer_id')[col.replace('is_','has_')].max()
    cust_train = cust_train[cust_train > 0].index
    
    #разобъем train на train/valid для валидации
    cust_train, cust_valid = train_test_split(cust_train, test_size = 0.1, shuffle = True, random_state = 2)
    print(cust_train.shape, cust_valid.shape)
    train = pd.DataFrame(cust_train, columns = ['customer_id']).merge(data_ca, how = 'left')
    valid = pd.DataFrame(cust_valid, columns = ['customer_id']).merge(data_ca, how = 'left')
    lgb_train = lgb.Dataset(train[xs] , train[col])
    lgb_eval = lgb.Dataset(valid[xs] , valid[col])
    print ("Training:", col)
    model[col] = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                verbose_eval=40,
                early_stopping_rounds=50)
    print ("Train accuracy:", score(train, ys = [col]))
    print ("Test accuracy:", score(valid, ys = [col]))
    print ()

    
# Train accuracy: 0.44998844998845
# Test accuracy: 0.44282744282744285

# Train accuracy: 0.35339894712748915
# Test accuracy: 0.3662551440329218
    

In [ ]:
(0.441403+0.333333)/2, 0.373875, (0.4276642+0.316872)/2

In [ ]:
cust_test = data_ca[data_ca['is_train'] == 0]['customer_id'].unique()
test = pd.DataFrame(cust_test, columns = ['customer_id']).merge(data_ca, how = 'left')
test = predict_proba(test)
test.rename(columns = {
        'customer_id':'_ID_',
        'is_home:add_lat': '_HOME_LAT_',
        'is_home:add_lon': '_HOME_LON_',
        'is_work:add_lat': '_WORK_LAT_',
        'is_work:add_lon': '_WORK_LON_'}, inplace = True)
test = test[['_ID_', '_WORK_LAT_', '_WORK_LON_', '_HOME_LAT_', '_HOME_LON_']]

In [ ]:
test3 = pd.read_csv('test_set.csv', dtype = dtypes, usecols = usecols_test)
submission = pd.DataFrame(test3['customer_id'].unique(), columns = ['_ID_'])


# Заполняем пропуски
submission = submission.merge(test, how = 'left').fillna(0)
submission.head()


In [ ]:
# Пишем файл submission
submission.to_csv('baseline-very-simple9.csv', index = False)

In [ ]:
# test = pd.DataFrame(cust_test, columns = ['customer_id']).merge(data_ca, how = 'left')
test.head()

In [ ]:
test2 = predict_proba(test)

test2.head()

In [ ]:
di = {}

for i in range(len(xs)):
    di[xs[i]] = model['is_home'].feature_importance()[i]
def my_plot_importance(booster, figsize, **kwargs): 
    from matplotlib import pyplot as plt
    from xgboost import plot_importance
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax, **kwargs)
%matplotlib inline
my_plot_importance(di, (10,20))